I need to fill the functions that I just used from the other files, create text explaining how to use the models, create the if statements for running the simulations or just plotting results from saved data. I think this may take 3 to 4 days, specially the explanation texts. After finishing this, I need to do a run of everything to make sure it works well. And them delete everything that it is not in this file, and put the dissertation original and the corrected version after I finishe subbitting the corrected version to the library with the links also for completeness of the repo, also put everything on zenodo and on github properly and with open code.

 # Imports

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt
import scipy as scp
from tqdm.auto import tqdm
from functools import partial
from IPython.display import display
from fractions import Fraction
from core.utils import *
from core.models import *
from core.plot_funcs import *
# Create more bools for running or just loading data
run_ks = True
run_opt= True

# Utils

## Visuals of the plots

In [ ]:
default_area=mpl.rcParams['figure.figsize'][0]*mpl.rcParams['figure.figsize'][1]
new_figsize=[14,8]
mpl.rcParams['figure.figsize'] = new_figsize
mpl.rcParams['lines.linewidth']=5
mpl.rcParams['xtick.minor.visible']=True
mpl.rcParams['ytick.minor.visible']=True
mpl.rcParams['axes.grid']=True
mpl.rc('xtick', labelsize=18)           # x-tick numbers only
mpl.rc('ytick', labelsize=18)           # y-tick numbers only
mpl.rcParams['figure.dpi']=300

## Comparative Tables

In [4]:
def compare_regions(data1, data2, regions, labels=["A", "B", "C"]):
    """
    Compare two datasets over given index regions.
    
    Parameters:
        data1, data2 : array-like
            The datasets to compare (must be same length).
        regions : list of tuples
            Each tuple (start, end) defines a region.
        labels : list of str
            Labels for the regions (must match len(regions)).
    
    Returns:
        summary : list of dict
            Each dict contains region label, mean diff, std dev.
    """
    summary = []

    for (start, end), label in zip(regions, labels):
        seg1 = np.array(data1[start:end])
        seg2 = np.array(data2[start:end])
        diff = seg1 - seg2
        mean_diff = np.mean(np.abs(diff))
        std_diff = np.std(diff)
        rmse = np.sqrt(np.mean(diff**2))
        summary.append({
            "region": label,
            "mean_abs_diff": mean_diff,
            "std_dev_diff": std_diff,
            "rmse": rmse,
            "max_abs_diff": np.max(np.abs(diff)),
            "min_abs_diff": np.min(np.abs(diff))
            
        })

    return summary
# To print as a simple table:
def print_summary_table(summary):
    print(f"{'Region':<8} {'Mean Abs Diff':<15} {'Std Dev Diff':<15} {'RMSE':<15} {'Max Diff':<15} {'Min Diff':<15}")
    for entry in summary:
        print(f"{entry['region']:<8} {entry['mean_abs_diff']:<15.6f} {entry['std_dev_diff']:<15.6f} {entry['rmse']:<15.6f} {entry['max_abs_diff']:<15.6f} {entry['min_abs_diff']:<15.6f}")

## Contrast

In [5]:
def contrast(dynamics,init_state,exp,b_v,w_p_v,w_p_r,omR,readtime=3.0):
    """
    Function find S0 S1 contrast the dynamics of the system.
    """
    _,res=dynamics(10.0,init_state,b=B0(b_v,0.0,0.0),w_p=w_p_v,om_r=omR,
                      om=D_gs-mu_e*b_v,ti=0.0,mode="Laser",progress_bar="OFF")
    a=res.states[-1]
    t=res.times[-1]
    _,res=dynamics(1.0,a,b=B0(b_v,0.0,0.0),w_p=w_p_v,om_r=omR,
                      om=D_gs-mu_e*b_v,ti=t,mode="Free",progress_bar="OFF")
    a=res.states[-1]
    t=res.times[-1]
    _,res=dynamics(10.0,a,b=B0(b_v,0.0,0.0),w_p=w_p_r,om_r=omR,
                      om=D_gs-mu_e*b_v,ti=t,mode="Laser",progress_bar="OFF")
    wind=int(5000*readtime/10)
    s0=np.sum(qt.expect(exp,res.states[:wind]))
    s0_int=scp.integrate.simpson(qt.expect(exp,res.states[:wind]),res.times[:wind])
    s0_full=np.sum(qt.expect(exp,res.states))
    s0_int_full=scp.integrate.simpson(qt.expect(exp,res.states),res.times)
    _,res=dynamics(np.pi/omR,a,b=B0(b_v,0.0,0.0),w_p=w_p_v,om_r=omR,
                      om=D_gs-mu_e*b_v,ti=t,mode="MW",progress_bar="OFF")
    a=res.states[-1]
    t=res.times[-1]
    _,res=dynamics(10.0,a,b=B0(b_v,0.0,0.0),w_p=w_p_r,om_r=omR,
                      om=D_gs-mu_e*b_v,ti=t,mode="Laser",progress_bar="OFF")
    s1=np.sum(qt.expect(exp,res.states[:wind]))
    s1_int=scp.integrate.simpson(qt.expect(exp,res.states[:wind]),res.times[:wind])
    s1_full=np.sum(qt.expect(exp,res.states))
    s1_int_full=scp.integrate.simpson(qt.expect(exp,res.states),res.times)
    return s0,s1,s0_full,s1_full,s0_int,s1_int,s0_int_full,s1_int_full

# Photoluminescence

## State Preparation